In [ ]:
using Plots
using LaTeXStrings
using Printf
using Random
using Statistics

In [ ]:
# recommended for readable figures
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfontsize=12, lw=2,ms=8)

# Direct Monte Carlo
Estimate
$$
I = \int_0^1 \cos(x/5) e^{-5x}dx
$$
by Direct Monte Carlo, as
$$
I = \frac{1}{n}\sum_{i=1}^n \cos(\eta_i/5) e^{-5\eta_i}
$$
with $\eta_i \sim U(0,1)$ i.i.d.

In [ ]:
g = x-> cos(x/5) * exp(-5*x);

In [ ]:
xx = LinRange(0,1,100)
plot(xx, g.(xx), label=L"$g(x)$")
plot!(xx, cos.(xx./5), label=L"$\cos(x/5)$")
xlabel!(L"$x$")

In [ ]:
Random.seed!(100)


n = 10^2;

n_trials = 10^2;

I_estimates = zeros(n_trials);

for k in 1:n_trials
    η_vals = rand(n); # generate the samples, rand samples U(0,1)
    I_estimates[k] = mean(g.(η_vals));
end


I_est = mean(I_estimates); # mean of the means averages
@printf("Mean estimate with %d samples = %g\n", n, I_est)
@printf("Variance of estimate with %d samples = %e\n", n, var(I_estimates))
histogram(I_estimates, label="Samples")
title!(latexstring(@sprintf("Estimates with \$n = %g\$",n)))
xlabel!(L"Estimate of $I$")

Relativley large spread in the histogram of estimates.

# Importance Sampler
Now estimate via importance sampling,
$$
I= I_1 \mathbb{E}[\cos(\xi/5)].
$$
Here,
$$
I_1 = \frac{1 - e^{-5}}{5}
$$
and $\eta$ has the pdf
$$
f_\xi(x) = \frac{e^{-5x}}{I_1}1_{[0,1]}(x)
$$

## Construct Inverse Function Sampler
This samples the $\xi$ variable from above.

In [ ]:
I₁ = (1-exp(-5))/5;
Finv =u-> -1/5 * log(1 - 5 * I₁ * u);
function ξ_sampler()
   return Finv(rand()); 
end

## Perform Importance Sampling

In [ ]:
Random.seed!(100)

G = x-> cos(x/5);

n = 10^2;

n_trials = 10^2;

I_estimates = zeros(n_trials);

for k in 1:n_trials
    ξ_vals = [ξ_sampler() for _ in 1:n]; # generate the samples
    I_estimates[k] = I₁ * mean(G.(ξ_vals));
end


I_est = mean(I_estimates); # mean of the means averages
@printf("Mean estimate with %d samples = %g\n", n, I_est)
@printf("Variance of estimate with %d samples = %e\n", n, var(I_estimates))
histogram(I_estimates, label="Samples")
title!(latexstring(@sprintf("Estimates with \$n = %g\$",n)))
xlabel!(L"Estimate of $I$")

Much more narrowly concentrated distribution